In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "/workspaces/mirrorverse/sqlite-jdbc-3.34.0.jar")
    .config(
        "spark.driver.extraClassPath",
        "/workspaces/mirrorverse/sqlite-jdbc-3.34.0.jar")
    .getOrCreate())

ps_conn = "jdbc:sqlite:/workspaces/mirrorverse/mirrorverse.db"

24/07/09 12:28:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from mirrorverse.models.chinook_depth.spark import build_training_data

depth_classes = [25, 50, 75, 100, 150, 200, 250, 300, 400, 500]
features = ["depth_class", "month", "daytime", "period_progress", "elevation"]
train_dir = "train"
test_dir = "test"

In [3]:
import numpy as np

build_training_data(
    spark,
    ps_conn,
    depth_classes=np.array(depth_classes),
    features=features,
    train_dir=train_dir,
    test_dir=test_dir,
    split=0.8,
    overwrite=True,
)

24/07/09 12:28:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
from mirrorverse.models.chinook_depth.keras import load_data, build_model
from tensorflow.keras.layers import Dense

def build_randomized_param_sets(param_grids, M, max_attempts):
    """
    Inputs:
    - param_grids: dict, parameter names to grid of values
    - M: int, number of parameter sets to generate
    - max_attempts: int, maximum number of attempts to generate a unique parameter set

    Outputs:
    - list of dicts, parameter sets
    """
    param_sets = []
    attempts = 0
    while len(param_sets) < M:
        assert attempts < max_attempts

        param_set = {}
        for param, grid in param_grids.items():
            param_set[param] = np.random.choice(grid)
        if param_set in param_sets:
            attempts += 1
        else:
            attempts = 0
            param_sets.append(param_set)

    return param_sets

grids = {
    "batch_size": [100, 500, 1000, 5000, 10000],
    "epochs": [10],
    "layers": [
        lambda: [Dense(64, activation='relu'), Dense(32, activation='relu'), Dense(16, activation='relu')],
        lambda: [Dense(32, activation='relu'), Dense(16, activation='relu'), Dense(8, activation='relu')],
        lambda: [Dense(16, activation='relu'), Dense(8, activation='relu'), Dense(4, activation='relu')],

        lambda: [Dense(16, activation='relu'), Dense(8, activation='relu'), Dense(16, activation='relu')],
        lambda: [Dense(8, activation='relu'), Dense(16, activation='relu'), Dense(8, activation='relu')],
        lambda: [Dense(16, activation='relu'), Dense(16, activation='relu'), Dense(16, activation='relu')],
    ]
}

param_sets = build_randomized_param_sets(grids, 20, 10)
param_sets


[{'batch_size': 500, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 500, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 5000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 10000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 100, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 500, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 1000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 1000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 10000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 5000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 10000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 1000, 'epochs': 10, 'layers': <function __main__.<lambda>()>},
 {'batch_size': 500, 'epochs': 10, 'layers': <functio

In [6]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("/chinook_depth_model_tuning")

# Turn off autologging.
mlflow.tensorflow.autolog(disable=True)

N = len(depth_classes)
for param_set in param_sets:
    batch_size = param_set["batch_size"]
    epochs = param_set["epochs"]
    layers = param_set["layers"]()

    with mlflow.start_run() as run:

        train = load_data(train_dir, N, features, batch_size=batch_size, shuffle_buffer_size=10000)
        test = load_data(test_dir, N, features, batch_size=batch_size, shuffle_buffer_size=10000)

        for i, layer in enumerate(layers):
            mlflow.log_param(f"layer_{i}_units", layer.get_config()['units'])

        model, layers = build_model(N, features, layers)

        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", epochs)
    
        history = model.fit(train, validation_data=test, epochs=epochs)

        for metric_name, metrics in history.history.items():
            for i, metric in enumerate(metrics):
                mlflow.log_metric(metric_name, metric, step=i)
            mlflow.log_metric(f"final_{metric_name}", metrics[-1])
            mlflow.log_metric(f"min_{metric_name}", min(metrics))

        mlflow.keras.log_model(model, "models")

2024/07/09 12:39:00 INFO mlflow.tracking.fluent: Experiment with name '/chinook_depth_model_tuning' does not exist. Creating a new experiment.


Epoch 1/10
   1649/Unknown 31s 18ms/step - loss: 1.5118

2024-07-09 12:39:31.493181: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/root/miniconda3/envs/tensorflow/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1667/1667 ━━━━━━━━━━━━━━━━━━━━ 38s 22ms/step - loss: 1.5111 - val_loss: 1.3645
Epoch 2/10


2024-07-09 12:39:38.425271: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


...

2024-07-09 15:11:53.622786: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


824/834 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 1.4374

2024-07-09 15:12:28.246963: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


834/834 ━━━━━━━━━━━━━━━━━━━━ 43s 51ms/step - loss: 1.4373 - val_loss: 1.3280


2024-07-09 15:12:36.610976: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024/07/09 15:12:36 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


In [9]:
from mirrorverse.models.chinook_depth.spark import run_inference

query = """
with h3 as (
    select distinct
        h3_level_4_key 
    from 
        tag_tracks
), time as (
    select distinct
        date_key as epoch
    from 
        dates
)
select 
    t.epoch,
    h3.h3_level_4_key
from 
    h3
    cross join time t
"""

run_inference(
    spark, 
    ps_conn,
    depth_classes,
    features,
    query,
    "train/normalization_parameters.json",
    "/workspaces/mirrorverse/mlartifacts/487865629624282725/12ca6296fce441ce88f96589a5b38a64/artifacts/models/data/model.keras",
    "depth_predictions"
)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step/ 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step
24/07/09 17:26:23 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step
313/313 ━━━━━━━━

In [10]:
import pandas as pd

pd_conn = "sqlite:////workspaces/mirrorverse/mirrorverse.db"
df = pd.read_sql("select * from depth_predictions where epoch > (1676246400 -(3600 * 24 * 365)) ", pd_conn)
df

,h3_level_4_key,epoch,depth_class,probability
0,594692656546709503,1644796800,25.0,0.290286
1,594692656546709503,1644883200,25.0,0.290286
2,594692656546709503,1644969600,25.0,0.290286
3,594692656546709503,1645056000,25.0,0.290286
4,594692656546709503,1645142400,25.0,0.290286
...,...,...,...,...
2457535,594988459534319615,1674777600,500.0,0.000010
2457536,594988459534319615,1675123200,500.0,0.000010
2457537,594988459534319615,1675641600,500.0,0.000005
2457538,594988459534319615,1675728000,500.0,0.000005


## Notes

```bash
apt-get update
apt install default-jre

curl -O https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar


Needed conda to get h5py installed correctly

mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh

~/miniconda3/bin/conda init bash
```